## ECE 478 Financial Signal Processing
## Pset1: Portoflio Analysis

### Preprocessing

Here we will:
1. Take the Farma and French Dataset 48 benchmark, which was acquired from [Kenneth R. French](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html), clean it, and split its daily returns for the 48 securities into 17 files, one for each year 2000-2016.
2. Aquire the S&P500 daily lows and highs from [Yahoo Finance](https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC) and compute the daily returns, organizing the data into years equivalent to the benchmark in step 1.
3. Aquire the USD 3 month LIBOR rate from [here](http://iborate.com/usd-libor/). Converte it into an effective daily rate and packaged these daily rates into files by years 2000 - 2016, similar to the steps above. 


We will begin with reading the Farma and French 48 benchmark dataset, which provides returns for 48 composite portfolios, each of which is made up of stocks from a different sector in the economy.

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

In [2]:
daily_data = pd.read_csv("../input/48_Industry_Portfolios_daily.CSV", low_memory=False)
monthly_data = pd.read_csv("../input/48_Industry_Portfolios_Wout_Div.csv")

daily_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/48_Industry_Portfolios_daily.CSV'

We see returns for 48 sectors(Argic, Food, etc.) as well as an unamed first column which gives us the date in a [Year][Month][Day] format.

Although we can't see it above, the data  actually appears twice, once for average value weighted returns and again for equally weighted returns. We will be interested in the equally weighted returns data between the years 2000 - 2016.

In [ ]:
daily_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
tmp = daily_data.index[daily_data['Date'] == '19260701'].tolist()
equally_weighted_daily_data = daily_data.loc[24267:]

equally_weighted_daily_data.set_index('Date', inplace=True) # set the date as the index
equally_weighted_daily_data.head()

We now seperate each year into its own dataset. Note that the year doesn't have to start on Jan 1st and end on Dec 31st, due to the possibility of these dates landing on weekends. In fact, we will say that there are 251 work days in a full year.

In [ ]:
def split_into_years(original_data, beg_year = 2000, amount_years = 17):
    list_of_new_datasets = []
    for i in range(amount_years):
        # due to weekends, need to make sure we start with correct index
        start_date = (beg_year+i)*(10**4) + 101
        while (str(start_date) not in original_data.index):
            start_date += 1

        end_date = (beg_year+i)*(10**4) + 1231
        while (str(end_date) not in original_data.index):
            end_date -= 1

        list_of_new_datasets.append(original_data.loc[str(start_date):str(end_date)])
    return list_of_new_datasets

In [ ]:
daily_return_by_year = split_into_years(equally_weighted_daily_data) # list of our new datasets 

We will now deal with the missing data, which the dataset mentions takes the value -99.9 or -999. Just in case, we will search for empty cells also.

In [ ]:
for i, data in enumerate(daily_return_by_year):
    print('Dataset of:', i+2000, end=' --- ')
    for col in data:
        count = (data[col] == ' -99.99').sum() + (data[col] == ' -999').sum() + data[col].isnull().sum()
        if count > 0:
            print(col + ':', count, end="; ")
    print()

Seems that we are okay on the missing data for now. To finish the preprocessing of the benchmark dataset, let us store each new dataset in a different file:

In [ ]:
for i, data in enumerate(daily_return_by_year):
    data.to_csv('48_IP_eq_w_daily_returns_' + str(i+2000) + '.csv')

We will now move on to the S&P500 returns and USD LIBOR over these same years, making sure to normalize the time scales to daily. Note that we are approximating this approximation by including weekeds in our count, while recognizing that there are no daily returns on those days.

In [ ]:
sp_daily = pd.read_csv("../input/dailySP500.CSV")
sp_daily.head()

In [ ]:
print('Values to impute in Open:', sp_daily['Open'].isnull().sum())
print('Values to impute in Adj Close:', sp_daily['Adj Close'].isnull().sum())

Our data is already clean, so let us move on to finding the daily returns with the formula:
$$ Daily \ \ Return = \frac{(Adj\ \ Close) - (Open)}{Open} $$
and then proceed to split up these returns into individual years.

In [ ]:
temp_sp_ID =  [ i.replace('-', '') for i in sp_daily['Date'] ]
temp_sp_returns = 100 * (sp_daily['Adj Close'] - sp_daily['Open'] ) / sp_daily['Open']

sp_returns = pd.DataFrame({'Date':  temp_sp_ID, 'Return': temp_sp_returns })
sp_returns.set_index('Date', inplace=True)

sp_returns_by_year = split_into_years(sp_returns)

for i, data in enumerate(sp_returns_by_year):
    data.to_csv('SP_daily_returns_' + str(i+2000) + '.csv')

For the USD LIBOR, we will first format the date identically to our benchmark and S&P500 returns and then we will convert the 3 month dollar LIBOR into an effective daily rate using the equation:
$$ R =  100 \big[ (1+ \frac{R_m}{100})^{1/N} - 1 \big] $$

where $R$ is the daily interest rate, $m$  is the amount of times daily interest will be compounded per 3 month period, and $R_m$ is the 3 month interest rate.

Assuming a 260 day work year, we can say that there there are $\frac{251}{4} = 62$ work days per 3-month period, and so in our case $ m = 62$.


In [ ]:
libor_daily = pd.read_csv("../input/LIBOR USD.csv")
libor_daily.head()

In [ ]:
temp_id = []
for i in libor_daily['Date']:
    temp = i.split('.')
    temp_id.append(temp[2] + temp[1] + temp[0])

libor_daily['Date'] = temp_id
libor_daily.head()

In [ ]:
print('Values to impute in 3M column:', libor_daily['3M'].isnull().sum())

In [ ]:
m = 62

temp_daily_rate = [100*( (1 + i/100)**(1/m) -1) for i in libor_daily['3M'] ]

libor_intr = pd.DataFrame({ 'Date': libor_daily['Date'], 'Effective Daily Interest': temp_daily_rate })
libor_intr.set_index('Date', inplace=True)

libor_intr_by_year = split_into_years(libor_intr)

for i, data in enumerate(libor_intr_by_year):
    data.to_csv('LIBOR_daily_interest_' + str(i+2000) + '.csv')

and thats it! The actual analysis of this data is conducted [here](https://www.kaggle.com/guybaryosef/portfolio-assesment).